In [1]:
import numpy as np
import pandas as pd
import math

# Importing Data

In [2]:
# import order report
order_report = pd.read_excel("Company X - Order Report.xlsx")

# import pincode zones
pin_zone = pd.read_excel("Company X - Pincode Zones.xlsx")

# import SKU Master
sku = pd.read_excel("Company X - SKU Master.xlsx")

# import invoice
invoice = pd.read_excel("Courier Company - Invoice.xlsx")

# import Rates
rates = pd.read_excel("Courier Company - Rates.xlsx")

### Checking missing values

In [3]:
print("Missing Values in order Report are :\n",order_report.isna().sum())
print("Missing Values in Pincode are :\n",pin_zone.isna().sum())
print("Missing Values in SKU are :\n",sku.isna().sum())
print("Missing Values in Invoice are :\n",invoice.isna().sum())
print("Missing Values in Rates are :\n",rates.isna().sum())

Missing Values in order Report are :
 ExternOrderNo           0
SKU                     0
Order Qty               0
Payment Mode            0
Item Price(Per Qty.)    0
dtype: int64
Missing Values in Pincode are :
 Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64
Missing Values in SKU are :
 SKU           0
Weight (g)    0
dtype: int64
Missing Values in Invoice are :
 AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64
Missing Values in Rates are :
 Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64


###  No missing values im all datasets

### Check for duplicates


In [4]:
print("Number of duplicate rows in order Report is :",order_report.duplicated().sum())
print("Number of duplicate rows in Pincode is :",pin_zone.duplicated().sum())
print("Number of duplicate rows in SKU is :",sku.duplicated().sum())
print("Number of duplicate rows in Invoice is :",invoice.duplicated().sum())
print("Number of duplicate rows in Rates is :",rates.duplicated().sum())

Number of duplicate rows in order Report is : 2
Number of duplicate rows in Pincode is : 16
Number of duplicate rows in SKU is : 1
Number of duplicate rows in Invoice is : 0
Number of duplicate rows in Rates is : 0


### Drop duplicate rows from all tables.

since we have Order Qty column, duplicate rows in Order Report are redundant 

In [5]:
# Dropping duplicate rows
order_report.drop_duplicates(inplace=True)
sku.drop_duplicates(inplace=True)
pin_zone.drop_duplicates(inplace=True)

### Add weight

In [6]:
# Add weight per item column to order report
data = order_report.merge(sku,how='left',on='SKU')

In [7]:
# total weight of item in KG
data['Total_Weight_by_X(kg)'] = data['Weight (g)']*data['Order Qty']/1000

# Total price of items
data['Amount_of_order'] = data['Item Price(Per Qty.)']*data['Order Qty']


In [8]:
# Drop unnecessary columns
data.drop(labels=['Order Qty','Item Price(Per Qty.)','Weight (g)','SKU'],axis=1,inplace=True)

In [9]:
# Total weight and Amount for each order
data = data.groupby(by=['ExternOrderNo','Payment Mode'],as_index=False).sum()
data.shape

(124, 4)

# Merging different datasets

#### Merge Invoice data with pincode datato find zone for X

In [10]:
# Lets merge invoice data
data = data.merge(invoice,how='left',left_on='ExternOrderNo',right_on='Order ID')


In [11]:
# merge invoice with pincode data
data = data.merge(pin_zone,how='left',on=['Warehouse Pincode','Customer Pincode'])

# rename zone columns from invoice data as zone_by_courier and pincode as zone_by_X
data = data.rename(columns={'Zone_x':'Zone_by_Courier','Zone_y':'Zone_by_X'})

###  Add rate

In [12]:
rates['Zone'].unique()

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

#### Change zone values to lowercase because values in our data are lowercase

In [13]:
# Convert Uppercase to lowercase
rates['Zone'] = rates['Zone'].str.lower()

In [14]:
data = data.merge(rates,how='left',left_on='Zone_by_X',right_on='Zone')

# Calculate Charges

####  Forward Fix charge is same

In [15]:
# Forward additional charge 
data['Forward_additional_charge'] = (data['Total_Weight_by_X(kg)']/data['Weight Slabs']).astype(int)*data['Forward Additional Weight Slab Charge']

In [16]:
# Fixed RTO charges for forward and rto order
def fixed_rto_charge(row):
    if row['Type of Shipment'] == 'Forward and RTO charges':
        return row['RTO Fixed Charge']
    elif row['Type of Shipment'] == 'Forward charges':
        return 0
    else:
        return None
    
data['RTO_fixed_charge'] = data.apply(fixed_rto_charge, axis=1)

In [17]:
# Additional RTO charges for forward and rto order
def additional_rto_charge(row):
    if row['Type of Shipment'] == 'Forward and RTO charges':
        return math.floor(row['Total_Weight_by_X(kg)']/row['Weight Slabs'])*row['RTO Additional Weight Slab Charge']
    elif row['Type of Shipment'] == 'Forward charges':
        return 0
    else:
        return None
    
data['RTO_additional_charge'] = data.apply(additional_rto_charge, axis=1)

In [18]:
# COD Charge
def cod_charge(row):
    if ((row['Payment Mode']=='COD') & (row['Amount_of_order']<=300)):
        return 15
    elif ((row['Payment Mode']=='COD') & (row['Amount_of_order']>300)):
        return row['Amount_of_order']*0.05
    else:
        return 0

data['COD_charge'] = data.apply(cod_charge,axis=1)


In [19]:
# Calculate total expected charge for delivery
data['Expected_charges'] = data['Forward Fixed Charge'] + data['Forward_additional_charge'] + data['RTO_fixed_charge'] + data['RTO_additional_charge'] + data['COD_charge']

In [20]:
# Difference between expected and billed charges
data['diff_charges'] =  data['Expected_charges'] - data['Billing Amount (Rs.)']

In [21]:
# Add column weight slab charged by courier for output

data['Weight_slabs_Courier'] = data['Zone_by_Courier'].replace(to_replace=['a','b','c','d','e'],value=[0.25,0.5,0.75,1.25,1.5])

In [22]:
# add column to check if courier company overcharged or undercharged
def is_correct(row):
    if row['diff_charges'] == 0:
        return 'X correctly charged'
    elif row['diff_charges'] > 0:
        return 'X Undercharged'
    else:
        return 'X Overcharged'
data['over_or_under_charged'] = data.apply(is_correct,axis=1)

In [23]:
data.columns

Index(['ExternOrderNo', 'Payment Mode', 'Total_Weight_by_X(kg)',
       'Amount_of_order', 'AWB Code', 'Order ID', 'Charged Weight',
       'Warehouse Pincode', 'Customer Pincode', 'Zone_by_Courier',
       'Type of Shipment', 'Billing Amount (Rs.)', 'Zone_by_X', 'Zone',
       'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge', 'Forward_additional_charge',
       'RTO_fixed_charge', 'RTO_additional_charge', 'COD_charge',
       'Expected_charges', 'diff_charges', 'Weight_slabs_Courier',
       'over_or_under_charged'],
      dtype='object')

# Output

In [24]:
output = data[['Order ID','AWB Code','Total_Weight_by_X(kg)','Weight Slabs','Charged Weight','Weight_slabs_Courier','Zone_by_X',
               'Zone_by_Courier','Expected_charges','Billing Amount (Rs.)','diff_charges']]

In [25]:
summary = pd.DataFrame(index=['Total orders where X has been correctly charged'
                              ,'Total Orders where X has been overcharged'
                              ,'Total Orders where X has been undercharged'
                             ], columns=['Count','Amount(Rs.)'])


summary['Count'] = [(data['diff_charges']==0).sum(),
                    (data['diff_charges']<0).sum(),
                    (data['diff_charges']>0).sum()]

summary['Amount(Rs.)'] = [data[data['diff_charges']==0]['Billing Amount (Rs.)'].sum()
                          ,abs(data[data['diff_charges']<0]['diff_charges'].sum())
                          ,data[data['diff_charges']>0]['diff_charges'].sum()]

## Export File

In [26]:
# output.to_excel("Output.xlsx")
# summary.to_excel("Summary.xlsx")

# Problems in data

In [27]:
# compare zone from invoice and pincode data

transposed_df = data[['Zone_by_Courier','Zone_by_X']].T
duplicate_columns = transposed_df.duplicated()
print("Are columns same:",invoice.duplicated().any())

mismatch_zone = data[data['Zone_by_Courier'] != data['Zone_by_X']].shape[0]
print("Mismatched entries :",mismatch_zone)

Are columns same: False
Mismatched entries : 65


### *65 entries of zone in invoice are not according to pincode data

In [28]:
# Comparing weight of order to weight chared by courier company
print("Are columns same:",(data['Total_Weight_by_X(kg)'] == data['Charged Weight']).all())

print("Mismatched entries :",(data['Total_Weight_by_X(kg)'] != data['Charged Weight']).sum())

Are columns same: False
Mismatched entries : 116


### *Total weight do not match Charged Weight by courier on **116** entries